In [14]:
from pathlib import Path
import json
from typing import Dict
from typing import List, Tuple, Any, Union
from pathlib import Path
import pandas as pd
import nltk
from transformers import EvalPrediction
#from preprocess import Preprocessor

In [15]:
def nltk_tokenize(text: str, filter_puncts: bool = True) -> List[str]:
    punctuations = set([ch for ch in "!\"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"])
    text = text.strip('"') # NOTE: remove the quotes first
    tokens = nltk.tokenize.word_tokenize(text)
    if filter_puncts:
        tokens = list(filter(lambda t: t not in punctuations, tokens))
    return tokens
    
def longestCommonSubsequence(text1: list, text2: list) -> int:
    if len(text2) > len(text1):
        text1, text2 = text2, text1

    lcs = [[0] * (len(text2) + 1) for _ in range(2)]
    for i in range(1, len(text1)+1):
        for j in range(1, len(text2)+1):
            if text1[i-1] == text2[j-1]:
                lcs[i % 2][j] = lcs[(i-1) % 2][j-1] + 1
            else:
                lcs[i % 2][j] = max(lcs[(i-1) % 2][j], lcs[i % 2][j-1])

    return lcs[len(text1) % 2][len(text2)]

def compute_lcs_score(pred: list, ans: list) -> float:
    intersection = longestCommonSubsequence(pred, ans)
    union = len(pred) + len(ans) - intersection
    if union == 0:
        return 0
    lcs_score = intersection / union
    if (lcs_score < 0) or (lcs_score) > 1:
        raise ValueError("LCS score must be between 0 and 1")
    return lcs_score

def compute_lcs_scores(pred_df: pd.DataFrame, ans_df: pd.DataFrame) -> pd.DataFrame:
    ids, qp_scores, rp_scores = list(), list(), list()
    for _, prow in pred_df.iterrows():
        pid, qp_pred, rp_pred = prow["id"], prow["q'"], prow["r'"]
        qp_pred, rp_pred = [nltk_tokenize(pred) for pred in [qp_pred, rp_pred]]
        ans_rows = ans_df[ans_df.id == pid]

        for _, arow in ans_rows.iterrows():
            qp_ans, rp_ans = arow["q'"], arow["r'"]
            qp_ans, rp_ans = [nltk_tokenize(ans) for ans in [qp_ans, rp_ans]]
            qp_score, rp_score = compute_lcs_score(qp_pred, qp_ans), compute_lcs_score(rp_pred, rp_ans)

            for item, l in zip([pid, qp_score, rp_score], [ids, qp_scores, rp_scores]):
                l.append(item)

    assert ids == ans_df.id.tolist()
    lcs_df = pd.DataFrame(data={
        "id": ids,
        "qp_scores": qp_scores,
        "rp_scores": rp_scores
    })
    return lcs_df

def compute_final_score(lcs_df: pd.DataFrame) -> float:
    lcs_df["total_scores"] = lcs_df["qp_scores"] + lcs_df["rp_scores"]
    max_scores = lcs_df.groupby("id")["total_scores"].max()
    final_score = max_scores.sum() / (2 * len(max_scores))
    if (final_score < 0) or (final_score > 1):
        raise ValueError("The final score must be between 0 and 1, please check the implementation.")
    return final_score

In [69]:
pred_df = pd.read_csv("./outputs/predict_test.csv", names=["id", "q'", "r'"], dtype=str)
ans_df = pd.read_csv("test_gold_622.csv", names=["id", "q'", "r'"], dtype=str)

for i in range(pred_df.shape[0]):
    tempq = str(pred_df.iloc[i]["q'"]).rsplit(' ', 1)[0]
    tempr = str(pred_df.iloc[i]["r'"]).rsplit(' ', 1)[0]
    pred_df.iloc[i]["q'"] = tempq
    pred_df.iloc[i]["r'"] = tempr
print(pred_df.iloc[5]["q'"])
print(pred_df.isnull().values.any())
print(ans_df.isnull().values.any())
pred_df = pred_df.fillna('')
print(pred_df.isnull().values.any())

So taking peoples rights away is an american value ? I think I will push for legislation to make cancer surgery illegal .
False
False
False


In [70]:
if len(pred_df) != len(ans_df.groupby("id").size()):
    raise ValueError("The prediction file must have the same number of rows as the number of unique IDs in the answer file")

lcs_df = compute_lcs_scores(pred_df, ans_df) # has len(ans_df) rows of lcs_q' and lcs_r'
final_score = compute_final_score(lcs_df) # derive the final score by "1/2N (\sum_i^N(max_j(score_q' + score_r')))"
print(f"Final test score: {final_score}")

Final test score: 0.754260262235276


In [ ]:
#### bert-large-uncased #####
#500: 0.5845899012911759
#1000: 0.5673322547362882
#1500: 0.618273462886458
#2000: 0.5929113627103975
#2500: 0.5836752600091908
#3000: 0.5987110975060392
#3500: 0.5981470868846762
#4000: 0.5981470868846762
#4500: 0.5907715976593922


In [20]:
a = 'hypermutation would most likely just end up killing itself .'
print(a.rsplit(' ', 1)[0])

hypermutation would most likely just end up killing itself


In [67]:
pred_df = pd.read_csv("./outputs/predict_test.csv", names=["id", "q'", "r'"], dtype=str)

for i in range(6):
    print(str(pred_df.iloc[i]["q'"]))
    tempq = str(pred_df.iloc[i]["q'"]).rsplit(' ', 1)[0]
    tempr = str(pred_df.iloc[i]["r'"]).rsplit(' ', 1)[0]
    pred_df.iloc[i]["q'"] = tempq
    pred_df.iloc[i]["r'"] = tempr


once again , you seem to support the killing of certain people .
alright how did evolution start ?
Do you not even get statutory sick pay ?
Why do you expect me to provide evidence when you never do ?
What has caused you to believe such things ?
So taking peoples rights away is an american value ? I think I will push for legislation to make cancer surgery illegal . After


In [68]:
print(pred_df.iloc[5]["q'"])

So taking peoples rights away is an american value ? I think I will push for legislation to make cancer surgery illegal .
